In [2]:
import pandas as pd
import nose.tools
# Write your imports here

# Data Tidying and Cleaning Lab
## Reading, tidying and cleaning data. Preparing data for exploration, mining, analysis and learning

### Problem 1. Read the dataset (2 points)
The dataset [here](http://archive.ics.uci.edu/ml/datasets/Auto+MPG) contains information about fuel consumption in different cars.

Click the "Data Folder" link and read `auto_mpg.data` into Python. You can download it, if you wish, or you can read it directly from the link.

Give meaningful (and "Pythonic") column names, as per the `auto_mpg.names` file:
1. mpg
2. cylinders
3. displacement
4. horsepower
5. weight
6. acceleration
7. model_year
8. origin
9. car_name

In [3]:
mpg_data = pd.read_fwf("../Data Tidying and Cleaning/auto-mpg.data", header = None)

In [4]:
mpg_data.columns = ["mpg","cylinders","displacement","horsepower","weight","acceleration","model_year","origin","car_name"]

In [5]:
nose.tools.assert_is_not_none(mpg_data)

Print the first 4 rows in the dataset to get a feel of what it looks like:

In [6]:
mpg_data.head(4)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,"""amc rebel sst"""


### Problem 2. Inspect the dataset (1 point)
Write a function which accepts a dataset and returns the number of observations and features in it, like so: 

``` 10 observations on 15 features```

Where 10 and 15 should be replaced with the real numbers. Test your function with the `auto_mpg` dataset.

Make sure the function works with other datasets (don't worry about "1 features" or "1 observations", just leave it as it is).

In [7]:
def observations_and_features(dataset):
    """
    Returns the number of observations and features in the provided dataset
    """
    observations = len(mpg_data.axes[0])
    features = len(mpg_data.axes[1])
    return "{} observations on {} features".format(observations, features)

In [8]:
print(observations_and_features(mpg_data))

398 observations on 9 features


Inspect the data types for each column.

In [9]:
mpg_data.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model_year        int64
origin            int64
car_name         object
dtype: object

### Problem 3. Correct errors (1 point)
The `horsepower` column looks strange. It's a string but it must be a floating-point number. Find out why this is so and convert it to floating-point number.

In [10]:
mpg_data['horsepower'] = pd.to_numeric(mpg_data['horsepower'],errors='coerce')

In [11]:
nose.tools.assert_equal(mpg_data.horsepower.dtype, "float64")

### Problem 4. Missing values: inspection (1 point)
We saw that the `horsepower` column contained null values. Display the rows which contain those values. Assign the resulting dataframe to the `unknown_hp` variable.

In [12]:
def get_unknown_hp(dataframe): 
    unknown_hp = mpg_data[mpg_data.horsepower.isnull()]
    unknown_hp = unknown_hp.horsepower
    return unknown_hp

In [13]:
cars_with_unknown_hp = get_unknown_hp(mpg_data)
print(cars_with_unknown_hp)

32    NaN
126   NaN
330   NaN
336   NaN
354   NaN
374   NaN
Name: horsepower, dtype: float64


### Problem 5. Missing data: correction (1 point)
It seems like the `NaN` values are a small fraction of all values. We can try one of several things:
* Remove them
* Replace them (e.g. with the mean power of all cars)
* Look up the models on the internet and try our best guess on the power

The third one is probably the best but the first one will suffice since these records are too few. Remove those values. Save the dataset in the same `mpg_data` variable. Ensure there are no more `NaN`s.

In [14]:
mpg_data = mpg_data.dropna()
mpg_data.isnull().values.any()

False

In [15]:
nose.tools.assert_equal(len(get_unknown_hp(mpg_data)), 0)

### Problem 6. Years of production (1 + 1 points)
Display all unique model years. Assign them to the variable `model_years`.

In [16]:
def get_unique_model_years(dataframe):
    model_years = mpg_data['model_year'].unique()
    return model_years

In [17]:
model_years = get_unique_model_years(mpg_data)
print(model_years)

[70 71 72 73 74 75 76 77 78 79 80 81 82]


These don't look so good. Convert them to real years, like `70 -> 1970, 71 -> 1971`. Replace the column values in the dataframe.

## We use .map to call the function for every element in the array instead of adding 1900 to the number , because every time we start the code it will add 1900 to the new year ---> 1970 ---> 3870 ...


In [26]:
mpg_data.model_year = mpg_data.model_year.map({
    70: 1970,
    71: 1971,
    72: 1972,
    73: 1973,
    74: 1974,
    75: 1975,
    76: 1976,
    77: 1977,
    78: 1978,
    79: 1979,
    80: 1980,
    81: 1981,
    82: 1982
})

In [27]:
model_years = get_unique_model_years(mpg_data)
print(model_years)

[nan]


### Problem 7. Exploration: low-power cars (1 point)
The data looks quite good now. Let's try some exploration.

Write a function to find the cars which have the smallest number of cylinders and print their model names. Return a list of car names.

In [19]:
def get_model_names_smallest_cylinders(dataframe):
    car_names = mpg_data.loc[mpg_data.cylinders <= mpg_data.cylinders.min()].car_name
    return car_names

In [20]:
car_names = get_model_names_smallest_cylinders(mpg_data)
print(car_names)
nose.tools.assert_true(car_names.shape == (4,) or car_names.shape == (4, 1))

71     "mazda rx2 coupe"
111          "maxda rx3"
243         "mazda rx-4"
334      "mazda rx-7 gs"
Name: car_name, dtype: object


### Problem 8. Exploration: correlations (1 point)
Finally, let's see some connections between variables. These are also called **correlations**.

Find how to calculate correlations between different columns using `pandas`.

**Hint:** The correlation function in `pandas` returns a `DataFrame` by default. You need only one value from it.

Create a function which accepts a dataframe and two columns and prints the correlation coefficient between those two columns.

In [21]:
def calculate_correlation(dataframe, first_column, second_column):
    correlation = mpg_data[first_column].corr(mpg_data[second_column])
    return correlation

In [22]:
hp_weight = calculate_correlation(mpg_data, "horsepower", "weight")
print("Horsepower:Weight correlation coefficient:", hp_weight)
nose.tools.assert_almost_equal(hp_weight, 0.864537737574, delta = 0.01)


Horsepower:Weight correlation coefficient: 0.8645377375741444
